In [2]:
from datetime import datetime, timedelta
# import cartopy.crs as ccrs
import os
import s2sphere as s2
import json
import pandas as pd
import plotly
import plotly.graph_objects as go
# from tqdm.notebook import tqdm
# os.environ["CARTOPY_USER_BACKGROUNDS"] = "./../BG"
import numpy as np

dset = "sandy"

In [3]:
def get_s2cell(lat, long, level):
    generator_obj = s2.CellId().from_lat_lng(s2.LatLng.from_degrees(lat, long)).parent(level)
    token = generator_obj.to_token()
    lat = generator_obj.to_lat_lng().lat().degrees
    lng = generator_obj.to_lat_lng().lng().degrees
    return (token, lat, lng)
get_s2cells = lambda df: pd.DataFrame(df.apply(lambda row: get_s2cell(row['lat_x'], row['lon_x'], 14), axis=1).tolist(), columns=['token', 'slat', 'slng'])


In [95]:
all_tweets = pd.read_pickle(f"../data/pickles/{dset}/cleanmoji.pkl")#.head(10000)
all_tweets['created_at_date'] = pd.to_datetime(all_tweets['created_at'], infer_datetime_format=True) 
all_tweets = all_tweets.merge(get_s2cells(all_tweets), left_index=True, right_index=True)
all_tweets.set_index('created_at_date', inplace=True)
counts = pd.melt(all_tweets[[f'Emoji_{i+1}' for i in range(5)]])['value'].value_counts()
plot = all_tweets.resample('1H').mean()[['slat', 'slng', 'Sent_1']]
track = pd.read_pickle(f"../data/pickles/{dset}/track.pkl")
track['TIME_'] = track['TIME_'].dt.tz_localize("UTC")
us_tweets = all_tweets[all_tweets['cc'] == "US"]

all_tweets['cc'].unique(), all_tweets.columns


(array(['US', 'IE', 'DO', 'BS', 'AR', 'AU', 'PR', 'NL', 'MX', 'GB', 'GH',
        'CA', 'HT', 'BR', 'ES', 'JM', 'GN', 'KY', 'DE', 'FR', 'ID', 'HN',
        'TH', 'CL', 'PE', 'CR', 'PA', 'PH', 'CO', 'MY', 'SE', 'BD', 'CH',
        'GP', 'ZA', 'GR', 'VE', 'VI', 'NG', 'SA', 'IN', 'JP', 'BE', 'PT',
        'CN', 'BB', 'LK', 'LB', 'DK', 'VN', 'IT'], dtype=object),
 Index(['full_text', 'created_at', 'id', 'lon_x', 'lat_x', 'lat_y', 'lon_y',
        'name', 'admin1', 'admin2', 'cc', 'Text', 'Top5%', 'Emoji_1', 'Emoji_2',
        'Emoji_3', 'Emoji_4', 'Emoji_5', 'Pct_1', 'Pct_2', 'Pct_3', 'Pct_4',
        'Pct_5', 'Sent_1', 'Sent_2', 'Sent_3', 'Sent_4', 'Sent_5', 'token',
        'slat', 'slng'],
       dtype='object'))

In [20]:
# emoji usage in nyc vs the world
ny_tweets = all_tweets[(all_tweets['admin1'] == 'New York') | (all_tweets['admin1'] == 'New Jersey')]
nny_tweets = all_tweets[(all_tweets['admin1'] != 'New York') & (all_tweets['admin1'] != 'New Jersey')]

In [101]:
counts = list()
for i, d in (enumerate(plot.index)):
    if i == len(plot.index) - 1:
        break
    n = plot.index[i+1] 
    tweets = nny_tweets[(nny_tweets.index < n) & (d <= nny_tweets.index)]
    cur_track = track[track['TIME_'] < n]
    count = pd.melt(tweets[[f'Emoji_{i+1}' for i in range(5)]])['value'].value_counts()
    
    counts.append(count)

In [102]:
df = pd.DataFrame(counts, index=plot.index[:-1]).fillna(0)

import plotly.express as px

In [103]:
fig = px.line(df[df.max().sort_values()[-15:-5].index], labels={
                     "created_at_date": "Date (UTC)",
                     "value": "Count (per hour)",
                    'variable': 'Most occuring Emojis'
#                      "species": "Species of Iris"
                 },title='Outside New York Emoji Hourly Counts')
fig.write_image(f"nnyscatter.svg",)

In [96]:
counts = list()
for i, d in (enumerate(plot.index)):
    if i == len(plot.index) - 1:
        break
    n = plot.index[i+1] 
    tweets = all_tweets[(all_tweets.index < n) & (d <= all_tweets.index)]
    count = pd.melt(tweets[[f'Emoji_{i+1}' for i in range(5)]])['value'].value_counts()
    print(count.index[count.argmax()])



😳
👍
👍
👍
👍
😳
😳
😳
😳
♥
👍
😬
♥
😳
👍
😳
👍
👍
😳
👍
😳
👍
👍
👍
👍
👍
👍
👍
👍
👍
👍
👍
👍
😳
😜
😜
😳
👍
♥
😳
👍
👍
👍
👍
👍
👍
👍
👍
👍
👍
👍
👍
😳
👍
👍
👍
😳
😳
😳
😳
👍
😳
👍
👍
👍
👍
👍
👍
👍
👍
👍
👍
👍
👍
👍
👍
👍
👍


ValueError: attempt to get argmax of an empty sequence